# Due to Some Reason i had to stop working on this project will restart it before buliding an application , will revise pytorch at one shot

# Pytorch at one shot(Regression Based Analysis)


In [3]:
#Import library
import numpy as np 
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import mlflow
import mlflow.pytorch


In [4]:
#Load data 
data=pd.read_csv('C:/Users/aditya/Desktop/2023/Deep Learning/ANN_PROJECT_2/Data/AP001.csv')

In [5]:
data.head()

,From Date,To Date,PM2.5 (ug/m3),PM10 (ug/m3),NO (ug/m3),NO2 (ug/m3),NOx (ppb),NH3 (ug/m3),SO2 (ug/m3),CO (mg/m3),...,Temp (degree C),RH (%),WS (m/s),WD (deg),SR (W/mt2),BP (mmHg),VWS (m/s),Xylene (ug/m3),RF (mm),AT (degree C)
0,2016-07-01 10:00:00,2016-07-01 11:00:00,10.67,39.0,17.67,39.2,32.33,7.07,6.60,0.48,...,33.43,71.67,2.30,226.33,123.67,NaN,-0.1,0.1,0.0,23.05
1,2016-07-01 11:00:00,2016-07-01 12:00:00,2.00,39.0,20.50,41.9,35.80,7.40,NaN,0.49,...,33.70,70.00,2.50,223.00,186.00,NaN,-0.1,0.1,0.0,NaN
2,2016-07-01 12:00:00,2016-07-01 13:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016-07-01 13:00:00,2016-07-01 14:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016-07-01 14:00:00,2016-07-01 15:00:00,20.50,50.0,15.40,43.6,32.78,6.35,6.38,0.47,...,33.57,63.50,1.88,223.00,240.50,NaN,-0.1,0.1,0.0,NaN


In [6]:
#Quick cleaning
def get_cleaned_data(data : pd.DataFrame)-> pd.DataFrame :
   
    mlflow.set_experiment("Data Cleaning Expirement") 
    with mlflow.start_run():

        mlflow.log_metric("Row_Count_Before_Cleaning",data.shape[0])
        mlflow.log_metric("Column_Count_Before_Cleaning",data.shape[1])
        data.drop(['From Date','To Date','BP (mmHg)'],axis=1,inplace=True)
        data.dropna(inplace=True,axis=0)
        
        mlflow.log_metric("Row_Count_After_Cleaning",data.shape[0])
        mlflow.log_metric("Column_Count_After_Cleaning",data.shape[1])
         

    return data


In [7]:
data_cleaned=get_cleaned_data(data)

In [8]:
data_cleaned.head()

,PM2.5 (ug/m3),PM10 (ug/m3),NO (ug/m3),NO2 (ug/m3),NOx (ppb),NH3 (ug/m3),SO2 (ug/m3),CO (mg/m3),Ozone (ug/m3),Benzene (ug/m3),Toluene (ug/m3),Temp (degree C),RH (%),WS (m/s),WD (deg),SR (W/mt2),VWS (m/s),Xylene (ug/m3),RF (mm),AT (degree C)
0,10.67,39.00,17.67,39.20,32.33,7.07,6.60,0.48,14.50,1.00,4.63,33.43,71.67,2.30,226.33,123.67,-0.1,0.10,0.0,23.05
5,15.25,59.50,24.30,45.12,40.12,6.65,6.53,0.51,6.60,0.77,5.33,33.55,63.75,1.40,213.25,152.25,-0.1,0.10,0.0,25.60
6,11.67,60.00,26.73,49.10,43.80,6.93,5.70,0.46,17.43,0.80,6.47,33.57,64.00,1.50,208.67,92.67,-0.1,0.23,0.0,25.70
7,11.75,57.50,19.10,46.33,36.82,8.12,6.23,0.44,19.98,0.70,6.40,33.48,64.00,1.95,222.75,35.00,-0.1,0.10,0.0,25.15
8,18.00,57.75,14.50,44.20,32.33,7.78,6.07,0.45,12.20,0.70,5.30,33.23,63.75,2.15,220.75,14.00,-0.1,0.10,0.0,24.90


In [9]:
X=data_cleaned.drop(['PM2.5 (ug/m3)'],axis=1)
y=data_cleaned['PM2.5 (ug/m3)']
# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [10]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
X_train

array([[-1.00939245,  0.68983168,  0.44781674, ..., -0.01592968,
        -0.11777914,  0.41795585],
       [-0.27422283, -0.60106141, -0.99221009, ..., -0.13108382,
        -0.11777914,  1.56325439],
       [ 2.51329531,  1.73668547,  3.55204686, ...,  0.32953275,
         1.04582594, -1.00379407],
       ...,
       [-0.41206713, -0.51812883, -0.5487025 , ..., -0.07350675,
        -0.11777914,  0.23694602],
       [ 0.16228413,  0.48793845,  0.57153372, ..., -0.07350675,
        -0.11777914,  1.84957903],
       [-0.14403654, -0.27884797, -0.08742876, ..., -0.17138777,
        -0.11777914, -1.33290284]])

In [33]:
y_train

2043     18.50
34732    15.75
3886     92.25
56168    91.50
14431    37.50
         ...  
15581    18.75
55354    39.00
47664    30.75
1204     19.75
21237    30.00
Name: PM2.5 (ug/m3), Length: 38272, dtype: float64

In [13]:
#Define a Neural Network
class RegressionNetwork(nn.Module):
    def __init__(self,input_size):
        super(RegressionNetwork, self).__init__()
        self.fc1=nn.Linear(input_size,128)
        self.relu=nn.ReLU()
        self.fc2=nn.Linear(128,1)

    def forward(self,x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x
    

input_size=X_train.shape[1]

model=RegressionNetwork(input_size)
criterion=nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer with learning rate 0.001

# Training the model
num_epochs = 500

for epoch in range(num_epochs):
    inputs = torch.tensor(X_train, dtype=torch.float32)
    labels = torch.tensor(y_train.values, dtype=torch.float32)

    optimizer.zero_grad()  # Clears the gradients of all optimized tensors
    outputs = model(inputs)  # Forward pass
    loss = criterion(outputs, labels.view(-1, 1))  # Calculate the loss
    loss.backward()  # Backward pass (compute gradients)
    optimizer.step()  # Update weights

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')





Epoch [10/500], Loss: 1368.1001
Epoch [20/500], Loss: 1341.5349
Epoch [30/500], Loss: 1313.2312
Epoch [40/500], Loss: 1281.7229
Epoch [50/500], Loss: 1245.8152
Epoch [60/500], Loss: 1204.6545
Epoch [70/500], Loss: 1157.7517
Epoch [80/500], Loss: 1105.0007
Epoch [90/500], Loss: 1046.6973
Epoch [100/500], Loss: 983.5643
Epoch [110/500], Loss: 916.6779
Epoch [120/500], Loss: 847.3409
Epoch [130/500], Loss: 777.0450
Epoch [140/500], Loss: 707.3663
Epoch [150/500], Loss: 639.8935
Epoch [160/500], Loss: 576.0783
Epoch [170/500], Loss: 517.1313
Epoch [180/500], Loss: 463.8961
Epoch [190/500], Loss: 416.8004
Epoch [200/500], Loss: 375.8586
Epoch [210/500], Loss: 340.7924
Epoch [220/500], Loss: 311.1206
Epoch [230/500], Loss: 286.3166
Epoch [240/500], Loss: 265.7613
Epoch [250/500], Loss: 248.8134
Epoch [260/500], Loss: 234.8478
Epoch [270/500], Loss: 223.2393
Epoch [280/500], Loss: 213.4643
Epoch [290/500], Loss: 205.0712
Epoch [300/500], Loss: 197.7323
Epoch [310/500], Loss: 191.2196
Epoch [3

In [14]:
# Log parameters and metrics to MLflow
mlflow.log_param("num_epochs", num_epochs)
mlflow.log_param("learning_rate", 0.001)

# Log metrics
mlflow.log_metric("final_loss", loss.item())


In [15]:
# Evaluate the model
with torch.no_grad():
    model.eval()
    test_inputs = torch.tensor(X_test, dtype=torch.float32)
    predictions = model(test_inputs).squeeze().numpy()


In [16]:
predictions

array([ 9.980333,  9.338976, 30.484184, ..., 36.950035,  8.210926,
       25.289974], dtype=float32)